In [1]:
import numpy as np
import pandas as pd
from utils import get_data, get_preds, get_table, plot_rsme, other_stats, add_intervals_to_test, plot_interval_check, plot_interval_sizes
np.random.seed(2024)
train_bos, test_bos = get_data(filepath="processed_data/full_data_bos.csv", size_train=1000, size_test=1000, train_tup=(2021, 2023))
train_bos.to_csv("processed_data/train_bos.csv")
test_bos.to_csv("processed_data/test_bos.csv")
# test_bos = pd.read_csv("processed_data/test_bos.csv")

train_nyc, test_nyc = get_data(filepath="processed_data/full_data_nyc.csv", size_train=1000, size_test=100, train_tup=(2021, 2022), test_tup=(2024, 2025))
train_nyc.to_csv("processed_data/train_nyc.csv")
test_nyc.to_csv("processed_data/test_nyc.csv")
# test_nyc = pd.read_csv("processed_data/test_nyc.csv")

In [2]:
model_info = [
    # ("no_curr_pace", "stan_results/params_bos1.csv", ['total_pace', 'prop'], ['beta.1', 'beta.2'], True),
    # ("with_curr_pace", "stan_results/params_bos2.csv", ['total_pace', 'curr_pace', 'prop'], ['beta.1', 'beta.2', 'beta.3'], True),
    # ("rstan2d", "stan_results/params_bos0d.csv", ["total_pace", "curr_pace", "prop"], ["beta[1]", "beta[2]", "beta[3]"], True),
    ("no_curr_pace", "stan_results/params_nyc1.csv", ['total_pace', 'prop'], ['beta.1', 'beta.2'], True),
    ("with_curr_pace", "stan_results/params_nyc2.csv", ['total_pace', 'curr_pace', 'prop'], ['beta.1', 'beta.2', 'beta.3'], True),
]
train, test = train_nyc, test_nyc # train_bos, test_bos

models = {name: pd.read_csv(path) for (name, path, feats, betas, pleft) in model_info}
mpreds = {name: get_preds(test, pd.read_csv(path), feats_lis=feats, beta_lis=betas, propleft=pleft, name=name)
           for (name, path, feats, betas, pleft) in model_info}

test2 = get_table(test, mpreds)
test2

,id,dist,curr_pace,total_pace,finish,age,gender,year,prop,propleft,male,propxcurr,malexage,extrap,no_curr_pace,with_curr_pace
0,24725,5K,5.197505,5.197505,5.509205,35,M,2024,0.118497,0.881503,1,0.615891,35,7.655300,16.360097,16.192127
1,24726,5K,5.208333,5.208333,5.504892,35,M,2024,0.118497,0.881503,1,0.617174,35,7.274000,15.943609,15.775477
2,24727,5K,5.208333,5.208333,5.494141,30,M,2024,0.118497,0.881503,1,0.617174,30,7.024000,15.693609,15.525477
3,24728,5K,5.197505,5.197505,5.485569,33,M,2024,0.118497,0.881503,1,0.615891,33,7.105300,15.810097,15.642127
4,24729,5K,5.202914,5.202914,5.458603,31,M,2024,0.118497,0.881503,1,0.616532,31,6.331317,15.018510,14.850459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,24820,40K,4.716981,4.743833,4.731442,28,M,2024,0.947980,0.052020,1,4.471602,28,-0.388233,0.332725,-0.390941
796,24821,40K,4.721435,4.730369,4.730381,36,M,2024,0.947980,0.052020,1,4.475824,36,0.000383,0.722601,-0.046603
797,24822,40K,4.621072,4.738214,4.729851,44,M,2024,0.947980,0.052020,1,4.380682,44,-0.262421,0.459063,-0.051339
798,24823,40K,4.599816,4.727015,4.728261,23,M,2024,0.947980,0.052020,1,4.360532,23,0.039204,0.761736,0.273966


In [3]:
# s_data =pd.DataFrame([[4, 0.2, .5], [3, 0.5, .75]], columns=["total_pace", "prop", "propleft"])
# get_preds(s_data, models['rstan2c'], feats_lis = ["total_pace", "prop"], beta_lis = ["beta[1]", "beta[2]"], full=True)

In [4]:
labels = ["no_curr_pace", "with_curr_pace", "extrap"]
rmse_table = plot_rsme(test2, labels, "all_errors")
rmse_table["no_curr_pace_"] = 1 - (rmse_table["no_curr_pace"] / rmse_table["extrap"])
rmse_table["with_curr_pace_"] = 1 - (rmse_table["with_curr_pace"] / rmse_table["extrap"])
rmse_table

File saved: analysis/all_errors.png


,no_curr_pace,with_curr_pace,extrap,no_curr_pace_,with_curr_pace_
dist,,,,,
5K,11.375625,11.229682,4.504683,-1.525289,-1.492891
10K,8.011466,7.630120,3.595993,-1.227887,-1.121840
15K,6.219334,5.840584,3.587036,-0.733836,-0.628248
20K,4.996419,4.632593,3.459314,-0.444338,-0.339165
25K,4.242170,3.925679,2.816584,-0.506140,-0.393773
30K,2.668014,2.107650,2.437752,-0.094457,0.135412
35K,1.592619,1.165597,1.651207,0.035482,0.294094
40K,0.616184,0.273913,0.575350,-0.070972,0.523919


In [5]:
other_stats(test2[labels], test2["finish"])

,no_curr_pace,with_curr_pace,extrap
0,5.971423,5.713850,3.061664
1,0.171934,0.241829,0.782317


In [6]:
mpreds2 = {name: (42195 / 60) / get_preds(test, pd.read_csv(path), feats_lis=feats, beta_lis=betas, propleft=pleft, name=name, full=True)
           for (name, path, feats, betas, pleft) in model_info}
# model_preds2["new1"] = (42195 / 60) / get_preds(test, pd.read_csv("stan_results/params_bos1.csv"), feats_lis=['total_pace', 'curr_pace', 'prop'], beta_lis=['beta.1', 'beta.2', 'beta.3'], propleft=True, full=True)

In [7]:
pred_names = labels[:2]
intervals_tbl = add_intervals_to_test(test2, mpreds2, pred_names)
i_check = plot_interval_check(intervals_tbl, pred_names)
i_sizes = plot_interval_sizes(intervals_tbl, pred_names)

File saved: analysis/interval_check.png
File saved: analysis/interval_sizes.png


<Figure size 4000x4000 with 0 Axes>

<Figure size 3000x2000 with 0 Axes>